# LOADING LIBRARIES

In [1]:
import open3d as o3d
import numpy as np
import os
import sys
import matplotlib.pyplot as plt 
import meshplot as mp
import copy
path = r"C:\Users\maths\Downloads\vigaentertainmentinternproblems\MergingPointCloudData\MergingPointCloudData\cam_{}_{}.ply"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Loading Point clouds

In [2]:
pcd_1_2 = o3d.io.read_point_cloud(path.format(1,2))
pcd_2_3 = o3d.io.read_point_cloud(path.format(2,3))
pcd_3_4 = o3d.io.read_point_cloud(path.format(3,4))

# Estimating Normals

In [3]:
voxel_size = 0.005
radius = voxel_size *2 
# pcd_1_2_down = pcd_1_2.voxel_down_sample(voxel_size=voxel_size)
# pcd_2_3_down = pcd_2_3.voxel_down_sample(voxel_size=voxel_size)
# pcd_3_4_down = pcd_3_4.voxel_down_sample(voxel_size=voxel_size)
pcd_1_2.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30))
pcd_2_3.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30))
pcd_3_4.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30))

In [4]:
# pcd_1_2 = o3d.io.read_point_cloud(r"C:\Users\maths\Downloads\vigaentertainmentinternproblems\MergingPointCloudData\MergingPointCloudData\cam_{}_{}.ply".format(1,2))
# pcd_1_2.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_1_2, depth=9)
# o3d.visualization.draw_geometries([pcd_1_2_down,pcd_2_3_down, pcd_3_4_down])
# o3d.visualization.draw_geometries([pcd_1_2_down])
# o3d.visualization.draw_geometries([pcd_2_3_down])
# o3d.visualization.draw_geometries([pcd_3_4_down])

In [5]:
# print(np.asarray(pcd_1_2_down.points))

In [6]:
x = np.asarray(pcd_1_2.points)
y= np.asarray(pcd_2_3.points)
z = np.asarray(pcd_3_4.points)
x.shape,y.shape,z.shape

((2061864, 3), (2070560, 3), (2069472, 3))

In [7]:
y = y[:len(x)]
x = x[:len(x)]
z = z[:len(x)]

In [8]:
y[:,2] = x[:,2]
# z[:,2] = x[:,2]

In [9]:
for i in range(len(x)):
    pcd_2_3.points[i] = y[i]

In [10]:
# pcd_1_2.estimate_normals()
# pcd_2_3.estimate_normals()
pcds_down = [pcd_1_2, pcd_2_3]
o3d.visualization.draw_geometries(pcds_down)

# Merging 1st and 2nd Point cloud

In [55]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds_down)):
    pcd_combined += pcds_down[point_id]
o3d.io.write_point_cloud("combined1_2.pcd", pcd_combined)
o3d.visualization.draw_geometries([pcd_combined])

In [56]:
o3d.visualization.draw_geometries([pcd_combined,pcd_3_4])

In [57]:
pcd_1_2

PointCloud with 2061864 points.

In [58]:
pcd_combined_test = o3d.geometry.PointCloud()
pcd_combined_test.points = o3d.utility.Vector3dVector([0.5*(pcd_1_2.points[i] + pcd_2_3.points[i]) for i in range(len(pcd_1_2.points))])

In [59]:
pcd_1_2.points[0] + pcd_2_3.points[0]

array([ -0.1745075,   0.497038 , -11.17796  ])

In [60]:
o3d.visualization.draw_geometries([pcd_combined, pcd_3_4])

In [61]:
pcd_3_4 = o3d.io.read_point_cloud(path.format(3,4))

In [62]:
z = np.asarray(pcd_3_4.points)

In [63]:
z[:,2] = z[:,2] + 1

In [64]:
for i in range(len(z)):
    pcd_3_4.points[i] = z[i]

In [65]:
o3d.visualization.draw_geometries([pcd_combined, pcd_3_4])

# Merging 3rd with the combined point cloud of 1 and 2

In [66]:
pcd_combined_final = o3d.geometry.PointCloud()
pcd_combined_final =  pcd_combined_final + pcd_combined + pcd_3_4
o3d.io.write_point_cloud("combined1_2_3.pcd", pcd_combined_final)
o3d.visualization.draw_geometries([pcd_combined_final])
pcd_combined_final.estimate_normals()

In [67]:
down_sampled_final = pcd_combined_final.voxel_down_sample(0.02)
down_sampled_final.estimate_normals()

In [68]:
o3d.visualization.draw_geometries([down_sampled_final])

# Using Ball Pivoting for mesh (Surface reconsturction)

In [70]:
radii = [0.02]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd_combined_final, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([rec_mesh])

# Saving...

In [74]:
o3d.io.write_triangle_mesh("combined1_2_3_using_ball_pivot.ply", rec_mesh)
o3d.visualization.draw_geometries([rec_mesh])

# Using Poisson Reconstruction for mesh (Surface reconstruction)

In [75]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        down_sampled_final, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 32171 / 32150
[Open3D DEBUG] #   Got kernel density: 0.303000 (s), 880.633 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] #     Got normal field: 0.299 (s), 885.672 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 9.681693e-05 / 3.114697e+00
[Open3D DEBUG] #       Finalized tree: 0.564 (s), 899.34 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] #  Set FEM constraints: 1.22 (s), 894.566 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] #Set point constraints: 0.0810001 (s), 877.473 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 674465 / 299720 / 471097
[Open3D DEBUG] Memory Usage: 877.473 MB
[Open3D DEBUG] # Linear system solved: 0.652 (s), 905.035 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] Got average: 0.0209999 (s), 902.75 (MB) / 965.984 (MB) / 3337 (MB)
[Open3D DEBUG] Iso-Value: 4.984513e-01 = 1.603568e+04 / 3.217100e+04
[Open3D DEBUG] #     

# Saving...

In [76]:
o3d.io.write_triangle_mesh("combined1_2_3_using_poisson.ply", mesh)
o3d.visualization.draw_geometries([mesh])

# Pairwise Registration (Not getting desired output)

In [11]:
# def pairwise_registration(source, target):
#     print("Apply point-to-plane ICP")
#     icp_coarse = o3d.pipelines.registration.registration_icp(
#         source, target, max_correspondence_distance_coarse, np.identity(4),
#         o3d.pipelines.registration.TransformationEstimationPointToPlane())
#     icp_fine = o3d.pipelines.registration.registration_icp(
#         source, target, max_correspondence_distance_fine,
#         icp_coarse.transformation,
#         o3d.pipelines.registration.TransformationEstimationPointToPlane())
#     transformation_icp = icp_fine.transformation
#     information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
#         source, target, max_correspondence_distance_fine,
#         icp_fine.transformation)
#     return transformation_icp, information_icp


# def full_registration(pcds, max_correspondence_distance_coarse,
#                       max_correspondence_distance_fine):
#     pose_graph = o3d.pipelines.registration.PoseGraph()
#     odometry = np.identity(4)
#     pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
#     n_pcds = len(pcds)
#     for source_id in range(n_pcds):
#         for target_id in range(source_id + 1, n_pcds):
#             transformation_icp, information_icp = pairwise_registration(
#                 pcds[source_id], pcds[target_id])
#             print("Build o3d.pipelines.registration.PoseGraph")
#             if target_id == source_id + 1:  # odometry case
#                 odometry = np.dot(transformation_icp, odometry)
#                 pose_graph.nodes.append(
#                     o3d.pipelines.registration.PoseGraphNode(
#                         np.linalg.inv(odometry)))
#                 pose_graph.edges.append(
#                     o3d.pipelines.registration.PoseGraphEdge(source_id,
#                                                              target_id,
#                                                              transformation_icp,
#                                                              information_icp,
#                                                              uncertain=False))
#             else:  # loop closure case
#                 pose_graph.edges.append(
#                     o3d.pipelines.registration.PoseGraphEdge(source_id,
#                                                              target_id,
#                                                              transformation_icp,
#                                                              information_icp,
#                                                              uncertain=True))
#     return pose_graph

In [12]:
# print("Full registration ...")
# max_correspondence_distance_coarse = voxel_size * 15
# max_correspondence_distance_fine = voxel_size * 1.5
# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     pose_graph = full_registration(pcds_down,
#                                    max_correspondence_distance_coarse,
#                                    max_correspondence_distance_fine)

In [13]:
# print("Optimizing PoseGraph ...")
# option = o3d.pipelines.registration.GlobalOptimizationOption(
#     max_correspondence_distance=max_correspondence_distance_fine,
#     edge_prune_threshold=0.25,
#     reference_node=0)
# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     o3d.pipelines.registration.global_optimization(
#         pose_graph,
#         o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
#         o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
#         option)

In [14]:
# print("Transform points and display")
# for point_id in range(len(pcds_down)):
#     print(pose_graph.nodes[point_id].pose)
#     pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
# o3d.visualization.draw_geometries(pcds_down)